# Data cleaning
This script cleans the data to prepare it for the quantitative and qualitative analyses.

## Variables and data preparation

In [1]:
import pandas as pd
import os
import yaml
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import semester_cleaning

In [2]:
# vars
attitudes = ["Interest", "Value", "Self-efficacy", "Responsibility"]
majors = ["CS/SWE major", "CS/SWE minor", "Other"]

In [3]:
# vars to eventually make into command line arguments
semester = "F2025"

Data is stored in the `\data` directory. 

In [4]:
# open all data folders
data_dir = os.path.join(project_root, 'data')
ee_file = "F2025_ee.csv"
cg_file = "F2025_cg.csv"
col_file = "F2025_cols.yaml"
time_file = "F2025_course_times.yaml"

ee_data_path = os.path.join(data_dir, 'raw', ee_file)
cg_data_path = os.path.join(data_dir, 'raw', cg_file)

In [5]:
ee_data = pd.read_csv(ee_data_path)
cg_data = pd.read_csv(cg_data_path)
data_all = [ee_data, cg_data] # to avoid repetition

## Data cleaning
### Remove and reformat columns
Remove strange no break space characters from column names

In [6]:
for d in data_all:
    d.columns = d.columns.str.replace("\u00A0", " ", regex=False)

Remove useless columns since survey is anonymous. These columns are empty anyways
- Name
- Email

In [7]:
for d in data_all:
    d.drop(columns=["Email", "Name"], inplace=True)

Give shorter descriptive names to columns. The new column names are stored in the YAML file, which can be modified if the survey questions change, for reusability purposes. Opening this file creates a dictionary mapping the old column names to the new and shortened ones. 

In [8]:
with open(os.path.join(data_dir, col_file), 'r') as yaml_file:
    new_cols = yaml.safe_load(yaml_file)

for d in data_all:
    d.rename(columns=new_cols, inplace=True)

### Merge the year columns into one single column
In the original data, the year column gets separated in two, one for undergrad and one for grad, since they are written as separate questions.

In [9]:
for d in data_all:
    d["Year undergraduate"] = d["Year undergraduate"].fillna(d["Year graduate"])
    d.drop(columns=["Year graduate"], inplace=True)
    d.rename(columns={"Year undergraduate": "Year"}, inplace=True)

### Format multiselect questions
Multiselect questions are one-hot encoded, since someone can select multiple answers.

#### Major
Note that the `dummies` function separates all possible responses to the question in its own column so we can take everything that is not a CS major/minor in the "Other" category. We take a sum since one can only have a 1 in one of the columns

In [10]:
for d in data_all:
    ohe_major = d["Major"].str.get_dummies(sep=";") # one hot encoded majors

    d["CS/SWE major"] = ohe_major["Computer Science or Software Engineering major"]
    d["CS/SWE minor"] = ohe_major["Computer Science or Software Engineering minor"]
    d["Other"] = ohe_major.drop(columns=["Computer Science or Software Engineering major", "Computer Science or Software Engineering minor"]).sum(axis=1)
    
    d.drop(columns=["Major"], inplace=True) # remove this line if we want to know exactly what the other majors are

#### EE course
This is for which EE course they are enrolled in

In [11]:
ohe_eecourse_ee = ee_data["EE course"].str.get_dummies(sep=";")
ohe_eecourse_cg = cg_data["EE course"].str.get_dummies(sep=";")
ohe_eecourse_all = [ohe_eecourse_ee, ohe_eecourse_cg]

eecourse_cols = list(ohe_eecourse_ee.columns) # this is the list of all EE courses
eecourse_cols_new = {}

# shorten the course column names in a general way
for col in eecourse_cols:
    course = col.split()[0]
    if "COMP" not in course:
        eecourse_cols_new[col] = "None"
    else:
        eecourse_cols_new[col] = course

for d_ohe in ohe_eecourse_all:
    d_ohe.rename(columns=eecourse_cols_new, inplace=True)

In [12]:
for i, d in enumerate(data_all):
    for col in ohe_eecourse_all[i].columns:
        d[col] = ohe_eecourse_all[i][col]
    d.drop(columns=["EE course"], inplace=True)

#### CG course
This is the control course students are in.

In [13]:
ohe_cgcourse = cg_data["CG course"].str.get_dummies(sep=";")

for col in ohe_cgcourse.columns:
    cg_data[col] = ohe_cgcourse[col]
cg_data.drop(columns=["CG course"], inplace=True)

### Remove whitespace from text entries

In [14]:
cols_with_text = ["Attitudes feedback", "EE summary", "Reasoning Q1", "Reasoning Q2", "Reasoning Q3"]
for d in data_all:
    for col in cols_with_text:
        d[col] = d[col].str.strip()

### Reorganize the data columns
Entries will have a hierarchy for easier access of data and generalizability

In [15]:
for i, d in enumerate(data_all):
    multi_cols = []

    for col in d.columns:

        # ee course
        if col in ohe_eecourse_all[i].columns:
            multi_cols.append(("EE course", col))

        # attitudes
        elif col.split()[0] in attitudes:
            multi_cols.append((col.split()[0], col))

        # major
        elif col in majors:
            multi_cols.append(("Field", col))

        # career
        elif "Career" in col:
            multi_cols.append(("Career", col))
            
        # reasoning Q
        elif "Reasoning" in col:
            multi_cols.append(("Reasoning", col))

        # EE specific - EE feedback
        elif "Lecture feedback" in col:
            multi_cols.append(("Lecture feedback", col))

        # EE specific - self assessment
        elif "Self-assessment" in col:
            multi_cols.append(("Self-assessment", col))

        # CG specific - CG course
        elif col in ohe_cgcourse.columns:
            multi_cols.append(("CG course", col))
    
        # the rest
        else:
            multi_cols.append((col, ""))
    d.columns = pd.MultiIndex.from_tuples(multi_cols)

### Format values

#### Likert scale questions
Convert to numerical values only

In [16]:
likert_map = {"1 - Strongly Disagree": 1,
              "2": 2,
              "3": 3,
              "4 - Neutral": 4,
              "5": 5,
              "6": 6,
              "7 - Strongly Agree": 7
             }

likert_cols = attitudes + ["Lecture feedback", "Self-assessment", "Career"]

In [17]:
for att in likert_cols:
    for col in ee_data[att].columns:
        ee_data[(att, col)] = ee_data[(att, col)].map(likert_map)

        # only attitudes for control group
        if "Lecture feedback" not in col and "Self-assessment" not in col:
            cg_data[(att, col)] = cg_data[(att, col)].map(likert_map)

Reverse Q5 of responsibility because the desired outcome is to disagree.

In [18]:
for data in data_all:
    data[("Responsibility", "Responsibility Q5")] = data[("Responsibility", "Responsibility Q5")] + 2*(4 - data[("Responsibility", "Responsibility Q5")])

#### Degree type
Shorten the statement of graduate students.

In [19]:
for d in data_all:
    d["Degree type"] = d["Degree type"].replace({"Graduate (masters or PhD)": "Graduate"})

#### Gender
Shorten the statement of genders that are not male or female.

In [20]:
for d in data_all:
    d["Gender"] = d["Gender"].replace({"Gender-fluid, non-binary, and/or two-spirit": "Other"})

#### Age
Shorten the age groups

In [21]:
age_map = {"Under 18": "<18",
           "Between 18-24": "18-24",
           "Between 25-34": "25-34",
           "Between 35-44": "35-44",
           "45 and above": ">45",
           "Prefer not to answer": "Prefer not to answer"}

for d in data_all:
    d["Age"] = d["Age"].map(age_map)

### Rename subcolumns to remove duplication of the targeted variable
E.g. "Reasoning Q1" --> "Q1"

In [22]:
likert_cols += ["Reasoning"]

for att in likert_cols:
    new_col_names = {}
    for col in ee_data[att].columns:
        new_name = col.split()[-1]
        new_col_names[col] = new_name
    ee_data.rename(columns=new_col_names, level=1, inplace=True)
    cg_data.rename(columns=new_col_names, level=1, inplace=True)

### Validate survey timing
We find the students who take the survey before the EE lecture occurred and remove their input for that course.

In [23]:
# convert times to datetimes
ee_data["Start time"] = pd.to_datetime(ee_data["Start time"])
ee_data["Completion time"] = pd.to_datetime(ee_data["Completion time"])

cg_data["Start time"] = pd.to_datetime(cg_data["Start time"])
cg_data["Completion time"] = pd.to_datetime(cg_data["Completion time"])

In [24]:
with open(os.path.join(data_dir, time_file), 'r') as yaml_file:
    course_times = yaml.safe_load(yaml_file)

for course in course_times:
    course_times[course] = pd.to_datetime(course_times[course])
    
    mask = (
        (ee_data[("EE course", course)] == 1) &
        (ee_data["Start time"] < course_times[course])
    ) # selects the columns for a single EE course and those whose times are earlier than the module time
    
    ee_data.loc[mask, ("EE course", course)] = 0

TODO: handle case where student is only on one course and answered survey before. In this case, we put in control

### Apply semester specific fixes
This includes mainly fixing the majors that are declared in the survey, and this must be done on a case by case basis. 

In [25]:
if semester == "F2025":
    semester_cleaning.F2025(ee_data, cg_data)

### Save new file that can be used for data analysis

In [26]:
clean_ee_file = ee_file.replace(".csv", "_clean.csv")
clean_cg_file = cg_file.replace(".csv", "_clean.csv")

ee_data.to_csv(os.path.join(data_dir, 'clean', clean_ee_file), index=False)
cg_data.to_csv(os.path.join(data_dir, 'clean', clean_cg_file), index=False)